In [50]:
import itertools
import numpy as np
import pandas as pd

from typing import List
from tqdm.notebook import tqdm

In [7]:
class Provider:

    def __init__(self, id: int, commission: float, conversion: float, processing_time: float):
        self.id = id
        
        self.commission = commission
        self.conversion = conversion
        self.processing_time = processing_time

    def __repr__(self):
        return f"P{self.id}"

In [95]:
payments = pd.read_csv("data/payments_1.csv")

payments.eventTimeRes = pd.to_datetime(payments.eventTimeRes)
payments = payments.sort_values(by="eventTimeRes")


providers = pd.read_csv("data/providers_1.csv")
providers.TIME = pd.to_datetime(providers.TIME)

providers = providers.sort_values(by="TIME")
providers = providers.drop_duplicates(subset=["TIME", "ID"], keep="last")

providers.index = range(len(providers))

In [10]:
def get_available_providers(transaction: pd.Series) -> List[Provider]:
    mask = (providers.CURRENCY == transaction["cur"]) & \
           (transaction["amount"] >= providers.MIN_SUM) & \
           (transaction["amount"] <= providers.MAX_SUM)
    
    available_providers = providers.copy()[mask].drop_duplicates("ID")
    
    providers_objects: List[Provider] = []
    for k, row in available_providers.drop_duplicates(subset=["ID"]).iterrows():
       providers_objects.append(
              Provider(
              id=k + 1,
              commission=row["COMMISSION"],
              conversion=row["CONVERSION"],
              processing_time=row["AVG_TIME"]
              )
       )

    return providers_objects

In [11]:
def compute_expected_processing_time(chain: List[Provider]) -> float:
    probas, values = [], []

    for k in range(len(chain)):
        multiplication = np.prod([1 - provider.conversion for provider in chain[:k]])

        probas.append(multiplication * chain[k].conversion)

        values.append(np.sum([provider.processing_time for provider in chain[:k + 1]]))

    probas, values = np.array(probas), np.array(values)

    return probas.dot(values)


def compute_expected_conversion(chain: List[Provider]) -> float:
    return 1 - np.prod([1 - provider.conversion for provider in chain])


def compute_expected_commission(chain: List[Provider]) -> float:
    probas, values = [], []

    for k in range(len(chain)):
        multiplication = np.prod([1 - provider.conversion for provider in chain[:k]])

        probas.append(multiplication * chain[k].conversion)
        values.append(chain[k].commission)

    probas, values = np.array(probas), np.array(values)

    return probas.dot(values)

In [43]:
all_available_providers = []

for k in tqdm(range(len(payments))):
    available_providers = get_available_providers(payments.iloc[k])

    all_available_providers.append(available_providers)

  0%|          | 0/193978 [00:00<?, ?it/s]

In [129]:
unique_providers_ids = set()
unique_providers = []

for provider_list in all_available_providers:
    for provider in provider_list:
        if provider.id not in unique_providers_ids:
            unique_providers.append(provider)
            unique_providers_ids.add(provider.id)

In [122]:
chain = all_available_providers[123021]
chain

[P29, P30, P31, P32]

In [103]:
values = []

for permutation in itertools.permutations(chain, r=len(chain)):
    permutation = list(permutation)

    values.append(
        (compute_expected_processing_time(permutation),
         compute_expected_commission(permutation),
         compute_expected_conversion(permutation))
    )

values = np.array(values)

In [133]:
permutation

[P32, P31, P30, P29]

In [105]:
values[:, 0].argmin(), values[:, 1].argmin()

(20, 4)

In [113]:
optimal_chain = list(itertools.permutations(chain, r=len(chain)))[4]
optimal_chain

(P29, P32, P30, P31)

In [118]:
optimal_chain[0].processing_time

18.0

In [119]:
optimal_chain[1].processing_time

14.0

In [120]:
optimal_chain[2].processing_time

16.0

In [121]:
optimal_chain[3].processing_time

26.0